# Loop data, Part 5: Plot day to day comparisons

What I'm trying to do this time:

- plot several days of data on top of each other
- have the user (well... me) select the number of days to show (I guess start and end date) and also select the time window of interest
- eventually (probably not in this notebook but we'll see) have some kind of hover tool (or something else??) to look at treatment info for the selected time period for a selected day. 

In [13]:
import sys
sys.path.append("../")

from mdb_tools import load_data as ld
from mdb_tools import schemas
from mdb_tools import loop_stats as oop
from mdb_tools import sugar_plots as sp

import pandas as pd
import numpy as np

from datetime import datetime
import pytz

import matplotlib.pyplot as plt
%matplotlib widget

In [14]:
yml_secrets_file = '../../secrets/mdb_secrets.yml'

# Access the database using the yml secrets file, and get a specific set of "collections"
col_entries, col_treatments, col_profile, col_devicestatus = ld.get_collections(yml_secrets_file)

# Grab schemas
entries_schema, treatments_schema, devicestatus_schema = schemas.mdb_schemas()

# Load
df_entries0 = col_entries.find_pandas_all({}, schema=entries_schema)
df_treatments = col_treatments.find_pandas_all({}, schema=treatments_schema)
df_devicestatus = col_devicestatus.find_pandas_all({}, schema=devicestatus_schema)

# Remove duplicate entries from cgm date - keep only loop for now.
df_entries = df_entries0[df_entries0["device"]=="loop://Dexcom/G6/21.0"].copy()

In [15]:
# Convert the time string to datetime format 
df_entries["time"] = pd.to_datetime(df_entries["dateString"]).dt.tz_convert('US/Eastern')
df_entries.set_index("time", inplace = True)
df_entries.sort_index(inplace=True)

In [16]:
# Set up my start/end dates and start/end times
date_start = '2023-10-26'
date_end = '2023-11-01'

# start and end time in hours/time of day
time_start = 0
time_end = 24

In [17]:
# Extract only the dates of interest
df_sub = df_entries.loc[date_start:date_end].copy()

# make a column of just hour of day
df_sub["hour"] = df_sub.index.hour
df_sub["date"] = df_sub.index.date
df_sub["time_of_day"] = df_sub.index.time

# Set up color scale 
unq_dates = np.unique(df_sub.date)
date_dict = dict(zip(unq_dates, range(len(unq_dates))))
df_sub["date_color"] = df_sub["date"].map(date_dict)

# Make another subset df that is just the requested hours
df_req = df_sub[(df_sub["hour"] >= time_start) & (df_sub["hour"] <= time_end)]

In [7]:
df_sub.head()

,sgv,dateString,date,device,hour,time_of_day,date_color
time,,,,,,,
2023-10-26 00:04:41-04:00,204.0,2023-10-26T04:04:41.000Z,2023-10-26,loop://Dexcom/G6/21.0,0,00:04:41,0
2023-10-26 00:09:40-04:00,204.0,2023-10-26T04:09:40.000Z,2023-10-26,loop://Dexcom/G6/21.0,0,00:09:40,0
2023-10-26 00:14:40-04:00,203.0,2023-10-26T04:14:40.000Z,2023-10-26,loop://Dexcom/G6/21.0,0,00:14:40,0
2023-10-26 00:19:40-04:00,203.0,2023-10-26T04:19:40.000Z,2023-10-26,loop://Dexcom/G6/21.0,0,00:19:40,0
2023-10-26 00:24:40-04:00,198.0,2023-10-26T04:24:40.000Z,2023-10-26,loop://Dexcom/G6/21.0,0,00:24:40,0


In [10]:
import hvplot.pandas

df_req.hvplot.line(x='time_of_day', y='sgv', by="date", line_color = "lightgray") * \
df_req.hvplot.points(x='time_of_day', y='sgv', by="date", color="date_color", cmap="Blues", 
                     ylabel="Blood glucose (mg/dL)", xlabel="Time of day")

:Overlay
   .NdOverlay.I  :NdOverlay   [date]
      :Curve   [time_of_day]   (sgv)
   .NdOverlay.II :NdOverlay   [date]
      :Points   [time_of_day,sgv]   (date_color)